In [ ]:
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
from ipydatagrid import DataGrid, TextRenderer
from IPython.display import display
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import minimize
from RiskMetrics import RiskAnalysis,diversification_constraint, create_constraint

In [ ]:
def objective(w):
    return np.sqrt(np.sum((w - w0) ** 2))
def sum_equal_one(weight):
    return np.sum(weight) - 1 

In [ ]:
file=pd.ExcelFile('2025.03.28 ULGOALS Mstar Breakdown.xlsx')
data_file=file.parse(sheet_name=file.sheet_names)

In [ ]:
holdings=data_file['Holdings'].set_index('Name')
holdings=holdings.loc[holdings.index!='Cash EUR']
holdings['Portfolio Weighting %']=holdings['Portfolio Weighting %']/holdings['Portfolio Weighting %'].sum()
sheets=file.sheet_names
sheets.remove('Holdings')
transparency={}
for sheet in sheets: 
    
    temp=data_file[sheet].set_index('Name').iloc[:,1:]
    temp=temp.loc[temp.index!='Cash EUR']
    temp=temp.loc[holdings.index]
    transparency[sheet]=temp/100

full_matrix=pd.DataFrame()
for key in transparency:
    full_matrix=pd.concat([full_matrix,transparency[key]],axis=1)

full_matrix

full_matrix_numpy=full_matrix.to_numpy()
w0=holdings['Portfolio Weighting %'].loc[full_matrix.index].to_numpy()

In [261]:
full_matrix

,Asset Alloc Cash %,Asset Alloc Equity %,Asset Alloc Bond %,Asset Alloc Other %,Value,Equity Style Core,Growth,Giant,Large,Mid,...,Thailand,Hong Kong,Philippines,Malaysia,Singapore,South Africa,Colombia,Chile,Vietnam,Other Countries
Name,,,,,,,,,,,,,,,,,,,,,
Amundi Fds Glb Eq ESG Improvers M2EURAcc,0.003917,0.995105,0.000000,0.000979,0.43331,0.43806,0.12863,0.48240,0.34586,0.12393,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
CPR Invest Climate Action I EUR Acc,0.012371,0.987613,0.000017,-0.000001,0.25469,0.55447,0.19084,0.41200,0.44188,0.14609,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
onemarkets Amundi Clmt Foc Eq E EUR Acc,0.007812,0.992318,0.000000,-0.000130,0.33622,0.42181,0.24197,0.42353,0.48756,0.07039,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
UBS ETF S&P 500 Scored & Scrn USD A-acc,0.000415,0.999585,0.000000,0.000000,0.32762,0.48608,0.18631,0.48627,0.34123,0.16650,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
CPR Invest Glbl Dsrpt Opp I EUR Acc,0.001130,0.994932,0.000220,0.003719,0.02629,0.43175,0.54196,0.42113,0.32326,0.22761,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
Amundi Fds New Silk Road ME EUR C,0.005244,0.999998,0.000270,-0.005512,0.14977,0.29349,0.55675,0.46892,0.32521,0.17334,...,0.000940,0.025800,0.007234,0.000000,0.011457,0.014416,0.0,0.001588,0.090896,0.055383
Fidelity Global Financial Svcs I-Acc-EUR,0.002791,0.984624,0.000162,0.012423,0.31592,0.47613,0.20795,0.50674,0.26083,0.17547,...,0.000000,0.003054,0.002520,0.003757,0.039734,0.000000,0.0,0.000000,0.000000,0.010731
Polar Capital Global Tech I Inc,0.016180,0.981240,0.002055,0.000525,0.00000,0.34638,0.65362,0.50752,0.22290,0.24817,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
Xtrackers MSCI World Con Staples ETF 1C,0.000000,1.000000,0.000000,0.000000,0.34894,0.63302,0.01803,0.43695,0.37616,0.18257,...,0.000000,0.001716,0.000000,0.000000,0.001086,0.000000,0.0,0.000000,0.000000,0.000000


In [287]:
data = pd.DataFrame({'Asset':[None],
'Sign':[None],
'Limit':[None]
})
drop_down_list_asset=list(full_matrix.index)+['All']
drop_down_list_sector=list(full_matrix.columns)
drop_down_list=drop_down_list_asset+drop_down_list_sector+[None]
constraints_options=["=", "≥", "≤"]

# Dropdown options
options1 = drop_down_list
options2 = constraints_options

# Create dropdown widgets
dropdown1 = widgets.Dropdown(description='Assets:', options=options1)
dropdown2 = widgets.Dropdown(description='Sign:', options=options2)
dropdown3 = widgets.FloatText(description='Limit')

# Button to submit row
submit_btn = widgets.Button(description='Add to DataFrame', button_style='success')

# Output area for showing dataframe
output = widgets.Output()

# Store data in a list of dicts
data = []

# Button callback
def on_submit_clicked(b):
    row = {
        'Asset': dropdown1.value,
        'Sign': dropdown2.value,
        'Limit': dropdown3.value
    }
    data.append(row)
    df = pd.DataFrame(data)
    display(df)


submit_btn.on_click(on_submit_clicked)

# Show the UI
display(widgets.VBox([dropdown1, dropdown2, dropdown3, submit_btn, output]))

In [289]:
display(DataGrid(pd.DataFrame(data)))

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

In [291]:
constraint_matrix=pd.DataFrame(data).to_numpy()
constraints=[]
dico_map = {'=': 'eq', '≥': 'ineq', '≤': 'ineq'}

try:
    for row in range(constraint_matrix.shape[0]):
        temp = constraint_matrix[row, :]
        ticker = temp[0]
        
        if ticker not in drop_down_list:
            continue
            
        sign = temp[1]
        limit = float(temp[2])

        if ticker=='All':
            constraint= diversification_constraint(sign,limit)
        
        elif ticker in drop_down_list_asset:
            position = np.where(full_matrix.index == ticker)[0][0]
            constraint = create_constraint(sign, limit, position)

        elif ticker in drop_down_list_sector:
            position=np.where(full_matrix.columns == ticker)[0][0]
            if sign=='≤' :
                constraints=[{'type': dico_map[sign], 'fun': lambda weights: limit-(weights@full_matrix_numpy)[position]}]
            elif sign=='≥' :
            
                constraints=[{'type': dico_map[sign], 'fun': lambda weights: (weights@full_matrix_numpy)[position]-limit}]
            else:
                constraints=[{'type': dico_map[sign], 'fun': lambda weights: (weights@full_matrix_numpy)[position]-limit}]
                
        constraints.extend(constraint)
        

except Exception as e:
    pass

In [293]:
bounds = [(0, 1) for _ in range(full_matrix.shape[0])]
result=minimize(objective, w0, method="SLSQP", bounds=bounds, constraints=constraints)
opt_weights = result.x
opt_weights.round(4)

array([0.    , 0.1155, 0.0984, 0.0873, 0.0699, 0.0623, 0.0621, 0.0431,
       0.0417, 0.0361, 0.0375, 0.028 , 0.0232, 0.0208, 0.0205, 0.0146,
       0.0152, 0.0152, 0.011 , 0.0101, 0.0018, 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    ])

In [339]:
initial,optimal=pd.DataFrame(w0,index=full_matrix.index,columns=["Initial"]),pd.DataFrame(opt_weights,index=full_matrix.index,columns=["Optimised"])
res=pd.concat([initial,optimal],axis=1).round(4)
res

,Initial,Optimised
Name,,
Amundi Fds Glb Eq ESG Improvers M2EURAcc,0.1541,0.0000
CPR Invest Climate Action I EUR Acc,0.1184,0.1155
onemarkets Amundi Clmt Foc Eq E EUR Acc,0.1012,0.0984
UBS ETF S&P 500 Scored & Scrn USD A-acc,0.0905,0.0873
CPR Invest Glbl Dsrpt Opp I EUR Acc,0.0745,0.0699
Amundi Fds New Silk Road ME EUR C,0.0648,0.0623
Fidelity Global Financial Svcs I-Acc-EUR,0.0621,0.0621
Polar Capital Global Tech I Inc,0.0494,0.0431
Xtrackers MSCI World Con Staples ETF 1C,0.0417,0.0417


In [351]:
(res.T@full_matrix).round(4).T.loc['Technology']

Initial      0.2559
Optimised    0.2000
Name: Technology, dtype: float64